In [17]:
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

## I was having issues with my acquire and prep files so I built everything from scratch using pandas
df = pd.read_csv("titanic.csv")
df

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
0,0,0,3,male,22.0,1,0,7.2500,S,Third,NaN,Southampton,0
1,1,1,1,female,38.0,1,0,71.2833,C,First,C,Cherbourg,0
2,2,1,3,female,26.0,0,0,7.9250,S,Third,NaN,Southampton,1
3,3,1,1,female,35.0,1,0,53.1000,S,First,C,Southampton,0
4,4,0,3,male,35.0,0,0,8.0500,S,Third,NaN,Southampton,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,886,0,2,male,27.0,0,0,13.0000,S,Second,NaN,Southampton,1
887,887,1,1,female,19.0,0,0,30.0000,S,First,B,Southampton,1
888,888,0,3,female,NaN,1,2,23.4500,S,Third,NaN,Southampton,0
889,889,1,1,male,26.0,0,0,30.0000,C,First,C,Cherbourg,1


In [18]:
df.drop_duplicates(inplace=True)
df.drop(columns=['deck', 'embarked', 'class', 'age'], inplace=True)
df.embark_town.fillna(value='Southampton', inplace=True)
dummy_df = pd.get_dummies(df[['sex', 'embark_town']], drop_first=True)
df = pd.concat([df, dummy_df], axis=1)
df.head()

,passenger_id,survived,pclass,sex,sibsp,parch,fare,embark_town,alone,sex_male,embark_town_Queenstown,embark_town_Southampton
0,0,0,3,male,1,0,7.2500,Southampton,0,1,0,1
1,1,1,1,female,1,0,71.2833,Cherbourg,0,0,0,0
2,2,1,3,female,0,0,7.9250,Southampton,1,0,0,1
3,3,1,1,female,1,0,53.1000,Southampton,0,0,0,1
4,4,0,3,male,0,0,8.0500,Southampton,1,1,0,1


In [19]:
train_validate, test = train_test_split(df, test_size=.2, random_state=123, stratify=df.survived)
train, validate = train_test_split(train_validate, 
                                       test_size=.3, 
                                       random_state=123, 
                                       stratify=train_validate.survived)

In [20]:
imputer = SimpleImputer(strategy='most_frequent')
train[['embark_town']] = imputer.fit_transform(train[['embark_town']])
validate[['embark_town']] = imputer.transform(validate[['embark_town']])
test[['embark_town']] = imputer.transform(test[['embark_town']])

In [106]:
test.head()

,passenger_id,survived,pclass,sex,sibsp,parch,fare,embark_town,alone,sex_male,embark_town_Queenstown,embark_town_Southampton
561,561,0,3,male,0,0,7.8958,Southampton,1,1,0,1
641,641,1,1,female,0,0,69.3000,Cherbourg,1,0,0,0
400,400,1,3,male,0,0,7.9250,Southampton,1,1,0,1
498,498,0,1,female,1,2,151.5500,Southampton,0,0,0,1
875,875,1,3,female,0,0,7.2250,Cherbourg,1,0,0,0


In [78]:
X = df.drop(['survived','sex', 'embark_town'],axis=1)
y = df[['survived']]

# Split into train, validate, and test datasets
X_train_validate, X_test, y_train_validate, y_test = train_test_split(X, y, test_size = .20, random_state = 123)
X_train, X_validate, y_train, y_validate = train_test_split(X_train_validate, y_train_validate, test_size = .30, random_state = 123)


X_train.head()

,passenger_id,pclass,sibsp,parch,fare,alone,sex_male,embark_town_Queenstown,embark_town_Southampton
689,689,1,0,1,211.3375,0,0,0,1
84,84,2,0,0,10.5000,1,0,0,1
738,738,3,0,0,7.8958,1,1,0,1
441,441,3,0,0,9.5000,1,1,0,1
643,643,3,0,0,56.4958,1,1,0,1


In [79]:
clf = DecisionTreeClassifier(max_depth=3, random_state=123)

In [80]:
clf.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=3, random_state=123)

In [81]:
y_pred = clf.predict(X_train)
y_pred[0:5]

array([1, 1, 0, 0, 0])

In [82]:
y_pred_proba = clf.predict_proba(X_train)
y_pred_proba

array([[0.01470588, 0.98529412],
       [0.12      , 0.88      ],
       [0.88732394, 0.11267606],
       [0.88732394, 0.11267606],
       [0.88732394, 0.11267606],
       [0.88732394, 0.11267606],
       [0.88732394, 0.11267606],
       [0.88732394, 0.11267606],
       [0.60714286, 0.39285714],
       [0.88732394, 0.11267606],
       [0.60714286, 0.39285714],
       [0.88732394, 0.11267606],
       [0.88732394, 0.11267606],
       [0.71428571, 0.28571429],
       [0.71428571, 0.28571429],
       [0.60714286, 0.39285714],
       [0.36      , 0.64      ],
       [0.68085106, 0.31914894],
       [0.12      , 0.88      ],
       [0.88732394, 0.11267606],
       [0.68085106, 0.31914894],
       [0.88732394, 0.11267606],
       [0.88732394, 0.11267606],
       [0.60714286, 0.39285714],
       [0.88732394, 0.11267606],
       [0.88732394, 0.11267606],
       [0.88732394, 0.11267606],
       [0.88732394, 0.11267606],
       [0.88732394, 0.11267606],
       [0.71428571, 0.28571429],
       [0.

In [83]:
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))

Accuracy of Decision Tree classifier on training set: 0.81


In [84]:
confusion_matrix(y_train, y_pred)

array([[280,  22],
       [ 71, 125]])

In [86]:
sorted(y_train.survived.unique())

[0, 1]

In [87]:
y_train.survived.value_counts()

0    302
1    196
Name: survived, dtype: int64

In [88]:
labels = sorted(y_train.survived.unique())

pd.DataFrame(confusion_matrix(y_train, y_pred), index=labels, columns=labels)

,0,1
0,280,22
1,71,125


In [89]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.93      0.86       302
           1       0.85      0.64      0.73       196

    accuracy                           0.81       498
   macro avg       0.82      0.78      0.79       498
weighted avg       0.82      0.81      0.81       498



In [90]:
print('Accuracy of Decision Tree classifier on test set: {:.2f}'
     .format(clf.score(X_validate, y_validate)))

Accuracy of Decision Tree classifier on test set: 0.78


In [95]:
clf = DecisionTreeClassifier(max_depth=5, random_state=123)

In [96]:
clf.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=5, random_state=123)

In [97]:
y_pred = clf.predict(X_train)
y_pred[0:5]

array([1, 1, 0, 0, 0])

In [98]:
y_pred_proba = clf.predict_proba(X_train)
y_pred_proba

array([[0.        , 1.        ],
       [0.25      , 0.75      ],
       [0.93006993, 0.06993007],
       [0.93006993, 0.06993007],
       [0.6       , 0.4       ],
       [0.93006993, 0.06993007],
       [0.86666667, 0.13333333],
       [0.93006993, 0.06993007],
       [0.23076923, 0.76923077],
       [0.93006993, 0.06993007],
       [0.23076923, 0.76923077],
       [0.86666667, 0.13333333],
       [0.2       , 0.8       ],
       [0.75      , 0.25      ],
       [1.        , 0.        ],
       [0.64705882, 0.35294118],
       [0.13333333, 0.86666667],
       [0.51851852, 0.48148148],
       [1.        , 0.        ],
       [0.93006993, 0.06993007],
       [0.51851852, 0.48148148],
       [0.93006993, 0.06993007],
       [0.93006993, 0.06993007],
       [0.64705882, 0.35294118],
       [0.86666667, 0.13333333],
       [0.93006993, 0.06993007],
       [0.93006993, 0.06993007],
       [0.93006993, 0.06993007],
       [0.93006993, 0.06993007],
       [1.        , 0.        ],
       [0.

In [99]:
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))

Accuracy of Decision Tree classifier on training set: 0.87


In [100]:
confusion_matrix(y_train, y_pred)

array([[293,   9],
       [ 57, 139]])

In [101]:
sorted(y_train.survived.unique())

[0, 1]

In [102]:
y_train.survived.value_counts()

0    302
1    196
Name: survived, dtype: int64

In [103]:
labels = sorted(y_train.survived.unique())

pd.DataFrame(confusion_matrix(y_train, y_pred), index=labels, columns=labels)

,0,1
0,293,9
1,57,139


In [104]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.97      0.90       302
           1       0.94      0.71      0.81       196

    accuracy                           0.87       498
   macro avg       0.89      0.84      0.85       498
weighted avg       0.88      0.87      0.86       498



In [105]:
print('Accuracy of Decision Tree classifier on test set: {:.2f}'
     .format(clf.score(X_validate, y_validate)))

Accuracy of Decision Tree classifier on test set: 0.78


The second set actually preformed better with the max_depth at 5, it reduced the type 1 and type 2 errors, but still ended up with the same accuracy in the end when predicting the likelyhood of survival on the titanic.